# Рекомендательные системы

Существуют различные подходы к построению рекомендательных систем: коллаборативный (User/Item Based kNN, Matrix factorization (ALS, SVD, FM, WR-MF, BPRMF), Neural collaborative filtering (NCF)), Learning to Rank (LTR: Pointwise, Pairwise(RankNet), Listwise(LambdaRank)), Sequential Recommendation (RNN, SASRec, FISSA), Collaborative Memory Networks (CMN), Collaborative Variational Autoencoder (CVAE), Reinforcement Learning (DDPG), Co-training recommendation, ... Многие описаны в хорошей обзорной статье на хабре ([часть 1](https://habr.com/ru/companies/prequel/articles/567648/), [часть 2](https://habr.com/ru/companies/prequel/articles/573880/)).

1. Выполните все задания в файле example.ipynb (см. example.zip) для данных example_data.csv. Добейтесь успешной проверки в конце файла.
2. Возьмите датасет вашего варианта (список см. ниже), сконвертируйте его в требуемую форму (см. example.zip) и запустите для него код файла example.ipynb. 
3. Запрограммируйте случайные рекомендации, а затем одинаковые для всех пользователей самые популярные. Составьте таблицу с двумя строчками: cold start, hot start, и четырьмя столбцами: random, popular, kNN, SVD. Заполните ее значениями метрики качества MAP@k для k=5 (или больше, если много нулевых значений).
4. Для метода kNN, cold_start=True и False для каждого пользователя из test выведите процент товаров, для которых предсказание рейтинга совпадает с kNN.default_prediction(). Что означают полученные результаты?
4. Возьмите одного пользователя из test для cold_start=True и выясните, отличаются ли первые k рекомендаций для него по методу kNN от самых популярных. Если да, то объясните почему.
5. Найдите пользователя из test для cold_start=False у которого значение details['actual_k'], возвращаемое функцией predict отлично от нуля. Найдите в списке его рекомендаций позиции товаров, которые покупали его соседи.

## [Варианты датасетов](https://cseweb.ucsd.edu/~jmcauley/datasets.html):
1. Twitch. [Описание](https://github.com/JRappaz/liverec). [Данные](https://drive.google.com/drive/folders/1BD8m7a8m7onaifZay05yYjaLxyVV40si)
2. [Food.com Recipe & Review Data](https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions)
3. [Amazon Product Reviews](https://cseweb.ucsd.edu/~jmcauley/datasets/amazon_v2/). Нужно выбрать только одну категорию из таблицы "Per-category data"
4. [Marketing Bias data](https://github.com/MengtingWan/marketBias). Выберите один из двух датасетов: ModCloth и Electronics
5. [Google Local Reviews (2021)](https://jiachengli1995.github.io/google/index.html). 
6. [Google Restaurants](https://www.kaggle.com/datasets/hwwang98/google-restaurants)
7. [Steam Video Game and Bundle Data](https://cseweb.ucsd.edu/~jmcauley/datasets.html#steam_data)
8. [Goodreads Book Reviews](https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home)
9. [Clothing Fit Data](https://cseweb.ucsd.edu/~jmcauley/datasets.html#clothing_fit)
10. [Product Exchange/Bartering Data](http://swapit.github.io/)
11. [Behance Community Art Data](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA)
12. [Social Recommendation Data](https://cseweb.ucsd.edu/~jmcauley/datasets.html#social_data)
13. [Social circles data](https://cseweb.ucsd.edu/~jmcauley/datasets.html#socialcircles). Нужно взять Google Plus
14. [MovieLens](https://grouplens.org/datasets/movielens/)
15. [WikiLens](https://grouplens.org/datasets/wikilens/)
16. [Book-Crossing](https://grouplens.org/datasets/book-crossing/)
17. [Book Genome Dataset](https://grouplens.org/datasets/book-genome/)
18. [Jester](https://goldberg.berkeley.edu/jester-data/)
19. [Rating Disposition](https://grouplens.org/datasets/rating-disposition-2023/)
20. [Learning from Sets of Items in Recommender Systems](https://grouplens.org/datasets/learning-from-sets-of-items-2019/)
21. [User personality and user satisfaction](https://grouplens.org/datasets/personality-2018/)
22. [Investigating Serendipity](https://grouplens.org/datasets/serendipity-2018/)
23. [Delicious Bookmarks](https://grouplens.org/datasets/hetrec-2011/)
24. [Last.FM](https://grouplens.org/datasets/hetrec-2011/)
